# Unrolled RNN
### Prediction Imputation

- Imputation with X_train, y_train
    - Divide into X_full and X_nan; y_full and y_nan
    - Use X_full and y_full to impute for X_nan and y_nan
- X_test, y_test
    - Impute with model learned from training data 

### Functions

In [266]:
from __future__ import print_function
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import tensorflow as tf
import sklearn as sk
from matplotlib import pyplot as plt

def select_columns(col_list, n_months):
    """
    Takes in a list of column names and number of visits starting at 0.
    Returns column list time-stepped and dovetailed.
    """ 
    return dovetail_names(*[time_step_names(i, n_months) for i in col_list])
        
def time_step_names(name, n_months):

    return [(name + '_%d' % (j+1)) for j in range(-1,n_months*6, 6)]

def dovetail_names(*kwargs):
    """
    Dovetails column names across time slices cccording to preset order.
    """
    zipped = zip(*kwargs)
    l = []
   
    for i in zipped:
        
        for j in i:
            
            l.append(j)
            
    return l

def reshape_data(X, y, n_time_steps, n_features = 110):  
    
    X_reshaped = X.values.reshape(-1, n_time_steps, n_features)
    y_reshaped = y.values.reshape(-1, n_time_steps, 1)
    print("X reshaped is " + str(X_reshaped.shape))
    print("y reshaped is " + str(y_reshaped.shape))

    y = y_reshaped.astype(float)
    X = X_reshaped.astype(float)
    
    print(X.shape, y.shape)
    return X, y

def provide_data(X,y,roll):
 
    X = X.iloc[:,:n_features*roll]
    y = y.iloc[:,:roll]

    y_full = y.dropna()

    mask = X.index.isin(y_full.index.tolist())

    X_full = X[mask]

    y_nan = y[~mask]
    X_nan = X[~mask]
    
    X_nan, y_nan = reshape_data(X_nan, y_nan, roll)
    X_full, y_full = reshape_data(X_full, y_full, roll)
    
    return X_full, X_nan, y_full, y_nan, mask
    

# Keras Callbacks

In [267]:
from keras import callbacks
import keras.layers as L
import keras.models as M
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error as mse

# reduce learning rate on plateau
reduce_lr = callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                               patience=5, min_lr=0.001)

# stop training if there isn't a significant improvement in the course of 5 epochs
early_stopping = callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, 
                              patience=5, verbose=0, mode='auto', 
                              baseline=None, restore_best_weights=True)


In [268]:
def start_model(X_train, y_train, X_test, y_test, n_time_steps, n_inputs=110):

    X_full_train, X_nan_train, y_full_train, y_nan_train, mask_train = provide_data(X_train, y_train, n_time_steps)
    X_full_test, X_nan_test, y_full_test, y_nan_test, mask_test = provide_data(X_test, y_test, n_time_steps)

    K.clear_session()

    m = M.Sequential()
    m.add(L.LSTM(n_units, input_shape=(n_time_steps, n_inputs), return_sequences=True))
    m.add(L.Dense(1, activation='relu'))
    m.compile(optimizer = 'nadam', loss = 'mean_squared_error')

    history = m.fit(X_full_train, y_full_train, batch_size = 50, 
                                  epochs=500,
                                  shuffle=True,
                   callbacks = [reduce_lr, early_stopping])

    col_name = X_train.columns.tolist()[(n_inputs*(n_time_steps+1))-1:(n_inputs*((n_time_steps+1)))]
    
    m_pred_train = pd.Series(np.array([x[0] for x in m.predict(X_nan_train)]).flatten(),
                             index = y_train[~mask_train].index)
    
    X_train.iloc[:,(n_inputs*(n_time_steps+1))-1].update(m_pred_train)
    y_train.iloc[:,n_time_steps-1].update(m_pred_train)
    
    m_pred_test = pd.Series(np.array([x[0] for x in m.predict(X_nan_test)]).flatten(),
                           index = y_test[~mask_test].index)
    
    X_test.iloc[:,(n_inputs*(n_time_steps+1))-1].update(m_pred_test)
    y_test.iloc[:,n_time_steps-1].update(m_pred_test)
    
    return X_train, y_train, X_test, y_test

In [269]:
def run_roll(X_train, y_train, X_test, y_test, r):
    
    for i in range(1,r+1):
        
        X_train, y_train, X_test, y_test = start_model(X_train,y_train, X_test, y_test,i)
        
    return X_train, y_train, X_test, y_test 

# Import Data

In [270]:
X = pd.read_csv("../limit_interp_data/X_6_months|6_months_limited_interpolation.csv", index_col = 0)
y = pd.read_csv("../limit_interp_data/y_6_months|6_months_limited_interpolation.csv", index_col = 0)

#n_time_steps = len(y.columns)
n_features = X.columns.tolist().index("EDSS_0")+1
pot = 1
n_inputs = n_features * pot
n_units = 64

print("The input length of the training data will be", pot, "time slices, separated by 6 month intervals")
print(n_features, "featues comprise one time slice")

The input length of the training data will be 1 time slices, separated by 6 month intervals
110 featues comprise one time slice


# Divide Data into Training and Testing

In [271]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

# Roll All the Way to 37

EDSS_222 has no full values!

In [272]:
X_train, y_train, X_test, y_test = run_roll(X_train, y_train, X_test, y_test, 37)

# X_train.to_csv('pred_data/X_train_6m|6m.csv')
# X_test.to_csv('pred_data/X_test_6m|6m.csv')
# y_train.to_csv('pred_data/y_train_6m|6m.csv')
# y_test.to_csv('pred_data/y_test_6m|6m.csv')

X reshaped is (60, 1, 110)
y reshaped is (60, 1, 1)
(60, 1, 110) (60, 1, 1)
X reshaped is (283, 1, 110)
y reshaped is (283, 1, 1)
(283, 1, 110) (283, 1, 1)
X reshaped is (17, 1, 110)
y reshaped is (17, 1, 1)
(17, 1, 110) (17, 1, 1)
X reshaped is (131, 1, 110)
y reshaped is (131, 1, 1)
(131, 1, 110) (131, 1, 1)
Epoch 1/500
283/283 [==============================] - 1s 5ms/step - loss: 3.8198
Epoch 2/500
283/283 [==============================] - 0s 146us/step - loss: 2.5032
Epoch 3/500
283/283 [==============================] - 0s 144us/step - loss: 1.9663
Epoch 4/500
283/283 [==============================] - 0s 152us/step - loss: 1.4887
Epoch 5/500
283/283 [==============================] - 0s 146us/step - loss: 1.1185
Epoch 6/500
283/283 [==============================] - 0s 147us/step - loss: 0.9095
Epoch 7/500
283/283 [==============================] - 0s 145us/step - loss: 0.7674
Epoch 8/500
283/283 [==============================] - 0s 148us/step - loss: 0.6749
Epoch 9/500
283/28

314/314 [==============================] - 0s 197us/step - loss: 0.2480
Epoch 26/500
314/314 [==============================] - 0s 204us/step - loss: 0.2342
Epoch 27/500
314/314 [==============================] - 0s 199us/step - loss: 0.2848
Epoch 28/500
314/314 [==============================] - 0s 194us/step - loss: 0.2182
Epoch 29/500
314/314 [==============================] - 0s 192us/step - loss: 0.1953
Epoch 30/500
314/314 [==============================] - 0s 199us/step - loss: 0.1707
Epoch 31/500
314/314 [==============================] - 0s 207us/step - loss: 0.1671
Epoch 32/500
314/314 [==============================] - 0s 201us/step - loss: 0.1565
Epoch 33/500
314/314 [==============================] - 0s 204us/step - loss: 0.2672
Epoch 34/500
314/314 [==============================] - 0s 199us/step - loss: 0.2546
Epoch 35/500
314/314 [==============================] - 0s 199us/step - loss: 0.1577
Epoch 36/500
314/314 [==============================] - 0s 196us/step - loss: 

Epoch 54/500
286/286 [==============================] - 0s 267us/step - loss: 0.1023
Epoch 55/500
286/286 [==============================] - 0s 240us/step - loss: 0.1090
Epoch 56/500
286/286 [==============================] - 0s 219us/step - loss: 0.0712
Epoch 57/500
286/286 [==============================] - 0s 212us/step - loss: 0.0665
Epoch 58/500
286/286 [==============================] - 0s 217us/step - loss: 0.0639
Epoch 59/500
286/286 [==============================] - 0s 212us/step - loss: 0.0621
Epoch 60/500
286/286 [==============================] - 0s 204us/step - loss: 0.0575
Epoch 61/500
286/286 [==============================] - 0s 220us/step - loss: 0.0551
Epoch 62/500
286/286 [==============================] - 0s 214us/step - loss: 0.0533
Epoch 63/500
286/286 [==============================] - 0s 215us/step - loss: 0.0503
Epoch 64/500
286/286 [==============================] - 0s 213us/step - loss: 0.0498
Epoch 65/500
286/286 [==============================] - 0s 208us/

299/299 [==============================] - 0s 287us/step - loss: 0.0482
Epoch 78/500
299/299 [==============================] - 0s 284us/step - loss: 0.0454
Epoch 79/500
299/299 [==============================] - 0s 289us/step - loss: 0.0446
Epoch 80/500
299/299 [==============================] - 0s 290us/step - loss: 0.0420
Epoch 81/500
299/299 [==============================] - 0s 285us/step - loss: 0.0462
Epoch 82/500
299/299 [==============================] - 0s 291us/step - loss: 0.0404
Epoch 83/500
299/299 [==============================] - 0s 313us/step - loss: 0.0424
Epoch 84/500
299/299 [==============================] - 0s 285us/step - loss: 0.0525
Epoch 85/500
299/299 [==============================] - 0s 284us/step - loss: 0.1113
Epoch 86/500
299/299 [==============================] - 0s 280us/step - loss: 0.0892
Epoch 87/500
299/299 [==============================] - 0s 284us/step - loss: 0.0480
X reshaped is (67, 5, 110)
y reshaped is (67, 5, 1)
(67, 5, 110) (67, 5, 1)
X 

291/291 [==============================] - 0s 363us/step - loss: 0.8213
Epoch 8/500
291/291 [==============================] - 0s 364us/step - loss: 0.7245
Epoch 9/500
291/291 [==============================] - 0s 368us/step - loss: 0.6851
Epoch 10/500
291/291 [==============================] - 0s 371us/step - loss: 0.6501
Epoch 11/500
291/291 [==============================] - 0s 364us/step - loss: 0.6100
Epoch 12/500
291/291 [==============================] - 0s 359us/step - loss: 0.6263
Epoch 13/500
291/291 [==============================] - 0s 364us/step - loss: 0.6142
Epoch 14/500
291/291 [==============================] - 0s 361us/step - loss: 0.5547
Epoch 15/500
291/291 [==============================] - 0s 366us/step - loss: 0.5351
Epoch 16/500
291/291 [==============================] - 0s 365us/step - loss: 0.6172
Epoch 17/500
291/291 [==============================] - 0s 378us/step - loss: 0.5315
Epoch 18/500
291/291 [==============================] - 0s 387us/step - loss: 0.

...

Epoch 23/500
27/27 [==============================] - 0s 2ms/step - loss: 1.3401
Epoch 24/500
27/27 [==============================] - 0s 2ms/step - loss: 1.3118
Epoch 25/500
27/27 [==============================] - 0s 2ms/step - loss: 1.2868
Epoch 26/500
27/27 [==============================] - 0s 2ms/step - loss: 1.2223
Epoch 27/500
27/27 [==============================] - 0s 2ms/step - loss: 1.1773
Epoch 28/500
27/27 [==============================] - 0s 2ms/step - loss: 1.1209
Epoch 29/500
27/27 [==============================] - 0s 2ms/step - loss: 1.1012
Epoch 30/500
27/27 [==============================] - 0s 2ms/step - loss: 1.0842
Epoch 31/500
27/27 [==============================] - 0s 2ms/step - loss: 1.0532
Epoch 32/500
27/27 [==============================] - 0s 2ms/step - loss: 0.9723
Epoch 33/500
27/27 [==============================] - 0s 2ms/step - loss: 0.9196
Epoch 34/500
27/27 [==============================] - 0s 2ms/step - loss: 0.9056
Epoch 35/500
27/27 [========

Epoch 36/500
13/13 [==============================] - 0s 5ms/step - loss: 1.4258
Epoch 37/500
13/13 [==============================] - 0s 6ms/step - loss: 1.4781
Epoch 38/500
13/13 [==============================] - 0s 6ms/step - loss: 1.4524
Epoch 39/500
13/13 [==============================] - 0s 5ms/step - loss: 1.2542
Epoch 40/500
13/13 [==============================] - 0s 5ms/step - loss: 1.1905
Epoch 41/500
13/13 [==============================] - 0s 5ms/step - loss: 1.1469
Epoch 42/500
13/13 [==============================] - 0s 5ms/step - loss: 1.1168
Epoch 43/500
13/13 [==============================] - 0s 5ms/step - loss: 1.0927
Epoch 44/500
13/13 [==============================] - 0s 6ms/step - loss: 1.0708
Epoch 45/500
13/13 [==============================] - 0s 5ms/step - loss: 1.0506
Epoch 46/500
13/13 [==============================] - 0s 5ms/step - loss: 1.0317
Epoch 47/500
13/13 [==============================] - 0s 5ms/step - loss: 1.0138
Epoch 48/500
13/13 [========

Epoch 46/500
4/4 [==============================] - 0s 12ms/step - loss: 1.7338
Epoch 47/500
4/4 [==============================] - 0s 11ms/step - loss: 1.7138
Epoch 48/500
4/4 [==============================] - 0s 11ms/step - loss: 1.6986
Epoch 49/500
4/4 [==============================] - 0s 12ms/step - loss: 1.6813
Epoch 50/500
4/4 [==============================] - 0s 12ms/step - loss: 1.6686
Epoch 51/500
4/4 [==============================] - 0s 13ms/step - loss: 1.6534
Epoch 52/500
4/4 [==============================] - 0s 11ms/step - loss: 1.6426
Epoch 53/500
4/4 [==============================] - 0s 11ms/step - loss: 1.6279
Epoch 54/500
4/4 [==============================] - 0s 12ms/step - loss: 1.6189
Epoch 55/500
4/4 [==============================] - 0s 12ms/step - loss: 1.6046
Epoch 56/500
4/4 [==============================] - 0s 12ms/step - loss: 1.5955
Epoch 57/500
4/4 [==============================] - 0s 11ms/step - loss: 1.5823
Epoch 58/500
4/4 [======================

Epoch 148/500
4/4 [==============================] - 0s 12ms/step - loss: 1.2997
Epoch 149/500
4/4 [==============================] - 0s 12ms/step - loss: 1.2976
Epoch 150/500
4/4 [==============================] - 0s 12ms/step - loss: 1.2975
Epoch 151/500
4/4 [==============================] - 0s 12ms/step - loss: 1.2957
Epoch 152/500
4/4 [==============================] - 0s 12ms/step - loss: 1.2956
Epoch 153/500
4/4 [==============================] - 0s 13ms/step - loss: 1.2940
Epoch 154/500
4/4 [==============================] - 0s 13ms/step - loss: 1.2940
Epoch 155/500
4/4 [==============================] - 0s 12ms/step - loss: 1.2924
Epoch 156/500
4/4 [==============================] - 0s 11ms/step - loss: 1.2925
Epoch 157/500
4/4 [==============================] - 0s 12ms/step - loss: 1.2910
Epoch 158/500
4/4 [==============================] - 0s 12ms/step - loss: 1.2911
Epoch 159/500
4/4 [==============================] - 0s 12ms/step - loss: 1.2895
Epoch 160/500
4/4 [=========

AttributeError: 'ProgbarLogger' object has no attribute 'log_values'

### Run Final Model for 6m|6m

In [349]:
X_train = pd.read_csv('pred_data/X_train_6m|6m.csv', index_col = 0)
y_train = pd.read_csv('pred_data/y_train_6m|6m.csv', index_col = 0)

X_test = pd.read_csv('pred_data/X_test_6m|6m.csv', index_col = 0)
y_test = pd.read_csv('pred_data/y_test_6m|6m.csv', index_col = 0)

X_train, y_train = reshape_data(X_train, y_train, 36)
X_test, y_test = reshape_data(X_test, y_test, 36)

In [355]:
K.clear_session()
n_units = 64
n_time_steps = 36
n_inputs = 110

m1 = M.Sequential()
m1.add(L.LSTM(64, input_shape=(n_time_steps, n_inputs), return_sequences=True))
m1.add(L.LSTM(32, return_sequences = True))
#m1.add(L.LSTM(16))
#m1.add(L.Dropout(0.5))
m1.add(L.Dense(1, activation='relu', input_dim = 3))
m1.compile(optimizer = 'nadam', loss = 'mean_squared_error')

history_m1 = m1.fit(X_train, y_train, batch_size = 50, epochs=500, 
                    validation_data=(X_test, y_test),
                   callbacks = [reduce_lr, early_stopping])                    

Train on 343 samples, validate on 148 samples
Epoch 1/500
343/343 [==============================] - 4s 12ms/step - loss: 2.8635 - val_loss: 1.9281
Epoch 2/500
343/343 [==============================] - 1s 3ms/step - loss: 1.6472 - val_loss: 1.7703
Epoch 3/500
343/343 [==============================] - 1s 3ms/step - loss: 1.3349 - val_loss: 1.1705
Epoch 4/500
343/343 [==============================] - 1s 3ms/step - loss: 1.1522 - val_loss: 1.0741
Epoch 5/500
343/343 [==============================] - 1s 3ms/step - loss: 0.9816 - val_loss: 1.0081
Epoch 6/500
343/343 [==============================] - 1s 3ms/step - loss: 1.0076 - val_loss: 1.0137
Epoch 7/500
343/343 [==============================] - 1s 3ms/step - loss: 0.8588 - val_loss: 0.8793
Epoch 8/500
343/343 [==============================] - 1s 3ms/step - loss: 0.8054 - val_loss: 0.8408
Epoch 9/500
343/343 [==============================] - 1s 3ms/step - loss: 0.7531 - val_loss: 0.8276
Epoch 10/500
343/343 [======================